In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import os
import shutil

### Lấy data và tạo thư mục chứa file csv

In [2]:
train_path = '../../data/raw/Activity/Train'
test_path = '../../data/raw/Activity/Test'

In [3]:
def rename_folders(dataset_path, mapping):
    for split in ['Train', 'Test']:  
        split_path = os.path.join(dataset_path, split)
        if not os.path.exists(split_path):
            print(f"Thư mục {split_path} không tồn tại.")
            continue

        # Đổi tên các thư mục theo mapping
        for old_name, new_name in mapping.items():
            old_folder = os.path.join(split_path, old_name)
            new_folder = os.path.join(split_path, new_name)
            if not os.path.exists(old_folder):
                if not os.path.exists(new_folder):
                    os.rename(old_folder, new_folder)
                    print(f"Đã đổi tên thư mục: {old_folder} -> {new_folder}")
                else:
                    print(f"Thư mục đích {new_folder} đã tồn tại. Bỏ qua việc đổi tên {old_folder}.")

        for class_folder in os.listdir(split_path):
            class_path = os.path.join(split_path, class_folder)

            if os.path.isdir(class_path):  # Kiểm tra nếu là thư mục
                videos = sorted(os.listdir(class_path))  # Lấy danh sách các video và sắp xếp theo thứ tự
                video_count = 1  # Đếm video bắt đầu từ 1

                for video in videos:
                    video_ext = os.path.splitext(video)[1]  # Lấy phần mở rộng của video (.mp4, .avi, ...)
                    old_video_path = os.path.join(class_path, video)
                    new_video_name = f"{class_folder}_{video_count}{video_ext}"  # Tạo tên mới theo dạng <Tên thư mục>_1.mp4
                    new_video_path = os.path.join(class_path, new_video_name)

                    if os.path.isfile(old_video_path):
                        os.rename(old_video_path, new_video_path)
                        print(f"Đã đổi tên video: {old_video_path} -> {new_video_path}")
                        video_count += 1
                    else:
                        print(f"File không hợp lệ: {old_video_path}")
            else:
                print(f"Thư mục {old_folder} không tồn tại. Bỏ qua.")


In [4]:
folder_mapping = {
    "Nhảy": "Jump",
    "Đấm": "Punch",
    "Đá": "Kick",
    "Sang trái": "Left",
    "Sang phải": "Right",
    "Đứng yên": "Stand"
    # Thêm các thư mục khác nếu cần
}

In [5]:
rename_folders("../../data/raw/Activity", folder_mapping)

Thư mục đích ../../data/raw/Activity/Train/Jump đã tồn tại. Bỏ qua việc đổi tên ../../data/raw/Activity/Train/Nhảy.
Thư mục đích ../../data/raw/Activity/Train/Punch đã tồn tại. Bỏ qua việc đổi tên ../../data/raw/Activity/Train/Đấm.
Thư mục đích ../../data/raw/Activity/Train/Kick đã tồn tại. Bỏ qua việc đổi tên ../../data/raw/Activity/Train/Đá.
Thư mục đích ../../data/raw/Activity/Train/Left đã tồn tại. Bỏ qua việc đổi tên ../../data/raw/Activity/Train/Sang trái.
Thư mục đích ../../data/raw/Activity/Train/Right đã tồn tại. Bỏ qua việc đổi tên ../../data/raw/Activity/Train/Sang phải.
Thư mục đích ../../data/raw/Activity/Train/Stand đã tồn tại. Bỏ qua việc đổi tên ../../data/raw/Activity/Train/Đứng yên.
Đã đổi tên video: ../../data/raw/Activity/Train/Stand/Stand_1.mp4 -> ../../data/raw/Activity/Train/Stand/Stand_1.mp4
Đã đổi tên video: ../../data/raw/Activity/Train/Stand/Stand_10.mp4 -> ../../data/raw/Activity/Train/Stand/Stand_2.mp4
Đã đổi tên video: ../../data/raw/Activity/Train/Stand/S

In [6]:
def copy_directory_structure(src, dst):
    # Duyệt qua toàn bộ cấu trúc thư mục của thư mục nguồn
    for dirpath, dirnames, filenames in os.walk(src):
        # Tạo đường dẫn mới tương ứng trong thư mục đích
        structure = os.path.join(dst, os.path.relpath(dirpath, src))
        if not os.path.isdir(structure):
            # Tạo thư mục nếu nó chưa tồn tại
            os.makedirs(structure)
            print(f"Created directory: {structure}")

In [7]:
# Đường dẫn tới thư mục nguồn
source_folder = '../../data/raw/Activity'

# Đường dẫn tới thư mục đích
destination_folder = '../../data/processed/CSV'

# Tạo cấu trúc thư mục giống hệt từ thư mục nguồn tới thư mục đích
copy_directory_structure(source_folder, destination_folder)

In [8]:
from moviepy.config import get_setting

print(get_setting("FFMPEG_BINARY"))  # Kiểm tra đường dẫn đến FFmpeg


/home/alexander/anaconda3/envs/Activity_Human_Recognition/lib/python3.10/site-packages/imageio_ffmpeg/binaries/ffmpeg-linux64-v4.2.2


In [9]:
from moviepy.editor import VideoFileClip
# Bước 2: Thay đổi kích thước video và lưu vào thư mục mới
def resize_videos_to_new_folder(dataset_path, output_path, target_size=(640, 640)):
    """
    Thay đổi kích thước video trong các thư mục con thành kích thước mục tiêu
    và lưu vào thư mục mới.

    :param dataset_path: Đường dẫn tới thư mục dataset (ví dụ: '/content/drive/MyDrive/Dataset DPL303m')
    :param output_path: Đường dẫn tới thư mục đầu ra để lưu video đã thay đổi kích thước
    :param target_size: Kích thước mục tiêu (width, height)
    """
    # Tạo thư mục đầu ra nếu chưa tồn tại
    os.makedirs(output_path, exist_ok=True)

    for split in ['Train', 'Test']:  # Duyệt qua cả 2 thư mục Train và Test
        split_path = os.path.join(dataset_path, split)
        if not os.path.exists(split_path):
            print(f"Thư mục {split_path} không tồn tại.")
            continue

        # Tạo thư mục đầu ra tương ứng với Train/Test
        output_split_path = os.path.join(output_path, split)
        os.makedirs(output_split_path, exist_ok=True)

        # Duyệt qua từng thư mục con trong Train/Test
        for class_folder in os.listdir(split_path):
            class_path = os.path.join(split_path, class_folder)

            if os.path.isdir(class_path):  # Kiểm tra nếu là thư mục
                # Tạo thư mục đầu ra cho từng lớp
                output_class_path = os.path.join(output_split_path, class_folder)
                os.makedirs(output_class_path, exist_ok=True)

                for video in os.listdir(class_path):
                    video_path = os.path.join(class_path, video)
                    video_ext = os.path.splitext(video)[1]

                    if os.path.isfile(video_path) and video_ext in ['.mp4', '.avi', '.mov']: 
                        try:
                            clip = VideoFileClip(video_path)
                            resized_clip = clip.resize(target_size)
                            new_video_name = f"resized_{video}"
                            new_video_path = os.path.join(output_class_path, new_video_name)
                            resized_clip.write_videofile(new_video_path, codec="libx264")
                            clip.close()
                            resized_clip.close()
                            print(f"Đã thay đổi kích thước video: {video_path} -> {new_video_path}")
                        except Exception as e:
                            print(f"Không thể xử lý video {video_path}: {e}")
                        break
                break

# Bước 3: Gọi hàm để thay đổi kích thước các video
dataset_folder = '../../data/raw/Activity'
output_folder = '../../data/processed/Resized_Videos'
resize_videos_to_new_folder(dataset_folder, output_folder)

Moviepy - Building video ../../data/processed/Resized_Videos/Train/Stand/resized_Stand_8.mp4.
MoviePy - Writing audio in resized_Stand_8TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ../../data/processed/Resized_Videos/Train/Stand/resized_Stand_8.mp4

Không thể xử lý video ../../data/raw/Activity/Train/Stand/Stand_8.mp4: must be real number, not NoneType
Moviepy - Building video ../../data/processed/Resized_Videos/Test/Stand/resized_Stand_1.mp4.
MoviePy - Writing audio in resized_Stand_1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ../../data/processed/Resized_Videos/Test/Stand/resized_Stand_1.mp4

Không thể xử lý video ../../data/raw/Activity/Test/Stand/Stand_1.mp4: must be real number, not NoneType


In [21]:
def extract_and_zip_landmarks(landmark_list, num_landmarks, prefix):
    landmarks = {}
    if landmark_list:
        for i, landmark in enumerate(landmark_list.landmark):
            landmarks[f"{prefix}_{i}"] = (landmark.x, landmark.y, landmark.z)
    else:
        for i in range(num_landmarks):
            landmarks[f"{prefix}_{i}"] = (np.nan, np.nan, np.nan)  # If no landmark, fill with NaN
    return landmarks

In [31]:
def model_function(video_path):
  # Initialize MediaPipe Holistic model and OpenCV
  mp_holistic = mp.solutions.holistic
  mp_drawing = mp.solutions.drawing_utils

  # Open the video file
  cap = cv2.VideoCapture(video_path)

  # Get video properties
  fps = cap.get(cv2.CAP_PROP_FPS)
  frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  duration = frame_count / fps
  frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

  # Initialize list to store results for each frame
  landmark_data = []

  # Initialize the Holistic model for detection
  with mp_holistic.Holistic(static_image_mode=False, model_complexity=1, 
                            enable_segmentation=False, refine_face_landmarks=False) as holistic:

      frame_num = 0

      while cap.isOpened():
          ret, frame = cap.read()

          if not ret:
              break
          # Calculate timestamp for current frame
          timestamp = frame_num / fps
          # Convert the image to RGB for MediaPipe
          image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
          # Process the image with MediaPipe Holistic model (without face detection)
          results = holistic.process(image)
          # Initialize row data with frame number and timestamp
          row_data = {'frame_num': frame_num, 'timestamp': timestamp}
          # Extract pose landmarks and zip (x, y, z)
          row_data.update(extract_and_zip_landmarks(results.pose_landmarks, 33, "pose"))
          # Extract left hand landmarks and zip (x, y, z)
          row_data.update(extract_and_zip_landmarks(results.left_hand_landmarks, 21, "left_hand"))
          # Extract right hand landmarks and zip (x, y, z)
          row_data.update(extract_and_zip_landmarks(results.right_hand_landmarks, 21, "right_hand"))
          # Append the row data to the list
          landmark_data.append(row_data)

          # Draw landmarks on the frame
          if results.pose_landmarks:
              mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
          if results.left_hand_landmarks:
              mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
          if results.right_hand_landmarks:
              mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
          # Move to the next frame
          frame_num += 1

    # Define the headers for the CSV file
  headers = ["frame_num", "timestamp"]

  # Add headers for each landmark (Pose, Face, Left Hand, Right Hand)
  pose_headers = [f"pose_{i}" for i in range(33)]
  left_hand_headers = [f"left_hand_{i}" for i in range(21)]
  right_hand_headers = [f"right_hand_{i}" for i in range(21)]

  # Combine all headers
  headers += pose_headers  + left_hand_headers + right_hand_headers

  df = pd.DataFrame(landmark_data, columns=headers)
  return df


In [28]:
from tqdm import tqdm

def process_videos_and_save_csv(source_root, destination_root):
    """
    Duyệt qua các video trong source_root, xử lý qua model_function,
    và lưu kết quả vào các file CSV trong destination_root.

    :param source_root: Đường dẫn thư mục nguồn chứa video
    :param destination_root: Đường dẫn thư mục đích để lưu CSV
    :param model_function: Hàm xử lý video và trả về DataFrame
    """
    # Duyệt qua tất cả các thư mục con trong source_root
    for dirpath, dirnames, filenames in os.walk(source_root):
        # Tính đường dẫn tương đối từ source_root
        relative_dir = os.path.relpath(dirpath, source_root)
        # Đường dẫn thư mục đích tương ứng
        dest_dir = os.path.join(destination_root, relative_dir)

        # Duyệt qua từng file trong thư mục hiện tại
        for filename in tqdm(filenames, desc=f"Processing {relative_dir}"):
            # Kiểm tra định dạng file video (có thể điều chỉnh theo định dạng bạn sử dụng)
            if filename.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                # Đường dẫn đầy đủ tới file video
                video_path = os.path.join(dirpath, filename)

                # Tạo tên file CSV tương ứng
                csv_filename = os.path.splitext(filename)[0] + '.csv'
                csv_path = os.path.join(dest_dir, csv_filename)

                # Xử lý video qua mô hình
                try:
                    df = model_function(video_path)
                    print(1)
                    # Lưu DataFrame vào file CSV
                    df.to_csv(csv_path, index=False)
                except Exception as e:
                    print(f"Lỗi khi xử lý {video_path}: {e}")


In [29]:
# Đường dẫn thư mục nguồn và đích
source_folder = '../../data/raw/Activity'
destination_folder = '../../data/processed/CSV'

# Gọi hàm xử lý
process_videos_and_save_csv(source_folder, destination_folder)

print("Đã hoàn thành xử lý tất cả các video.")


Processing .: 0it [00:00, ?it/s]
Processing Train: 0it [00:00, ?it/s]
Processing Train/Stand:   0%|          | 0/28 [00:00<?, ?it/s]libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open zink: /usr/lib/dri/zink_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr

1


libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open zink: /usr/lib/dri/zink_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: 

1


libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open zink: /usr/lib/dri/zink_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: 

...

1


libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open zink: /usr/lib/dri/zink_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: 

1


libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open zink: /usr/lib/dri/zink_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: 

KeyboardInterrupt: 

In [ ]:
shutil.copytree('/content/Data_csv', 'Dataset/CSV')

'/content/drive/MyDrive/Dataset DPL303m/CSV'